### Creating swap routes 

We are going to try to create the swap routes based on the swaps that we get out of the solver. The idea is to figure out what the path is of every token in this network from the set of swaps. One thing we have to watch out for is if things are cycling or something. But we will check. 

1. Set up the solver and stuff so that we can get the outputs
2. Go through the output swaps and create the paths . 

In [2]:
import pandas as pd 

from tqdm import tqdm
from test_stephon import (
    OrderRoutingSimulationEnvironment, 
    Swap, 
    solve_with_known_eta, solve_with_unknown_eta, 
    get_tokens_from_trade, 
    populate_chain_dict
) 


In [3]:
chains: dict[str, list[str]] = {
"ETHEREUM": ["WETH", "USDC", "SHIBA"],
'CENTAURI': [],
"OSMOSIS": ["ATOM","SCRT"],
}


In [4]:
chains: dict[str, list[str]] = {
"ETHEREUM": ["WETH", "USDC", "SHIBA"],
'CENTAURI': [],
"OSMOSIS": ["ATOM","SCRT"],
}

sim_env = OrderRoutingSimulationEnvironment(
    center_node='CENTAURI', 
    max_reserve=1e10, 
    chains=chains
)

# Solving with unknnown eta values
origin_token = "WETH"
number_of_init_tokens = 2000 
obj_token = "ATOM"


deltas, lambdas, psi, eta = solve_with_unknown_eta(
    origin_token, 
    number_of_init_tokens, 
    obj_token, 
    sim_env.all_tokens, 
    sim_env.all_cfmms, 
    sim_env.mapping_matrices
)

# We go through various values of eta and solve the problem for each of them
t_values = sorted([eta[i].value for i in range(len(sim_env.all_cfmms))])    
results = {} 

for j in tqdm(range(len(t_values))):
    example_eta = [int(eta[i].value >= t_values[j]) for i in range(len(sim_env.all_cfmms))]

    try: 
        deltas, lambdas, psi, objective = solve_with_known_eta(
            origin_token, 
            number_of_init_tokens, 
            obj_token, 
            sim_env.all_tokens, 
            sim_env.all_cfmms, 
            sim_env.mapping_matrices,
            example_eta
        )
        
        results[j] = {
            'deltas': [delta.value for delta in deltas],
            'lambdas': [lambda_.value for lambda_ in lambdas],
            'psi': psi.value,
            'objective': objective.value, 
            'eta': example_eta,
        }
        
    except:
        print(f"Failed for t={t_values[j]}")
        continue
    
# Getting the best result by looking at the objective value
best_result_key = max(results, key=lambda x: results[x]['objective'])
best_result = results[best_result_key]

# Taking the best result and getting the trades
best_eta = best_result['eta']
best_deltas = best_result['deltas']
best_lambdas = best_result['lambdas']
best_psi = best_result['psi']

chosen_cfmms = [cfmm for i, cfmm in enumerate(sim_env.all_cfmms) if best_eta[i] == 1]
chosen_mapping_matrices = [mapping_matrix for i, mapping_matrix in enumerate(sim_env.mapping_matrices) if best_eta[i] == 1]
chosen_deltas = [delta for i, delta in enumerate(best_deltas) if best_eta[i] == 1]
chosen_lambdas = [lambda_ for i, lambda_ in enumerate(best_lambdas) if best_eta[i] == 1]

# Creating the swaps from each of these
swaps = [] 

for i in range(len(chosen_cfmms)): 
    c = chosen_cfmms[i]
    m = chosen_mapping_matrices[i]
    d = chosen_deltas[i]
    l = chosen_lambdas[i]
    
    trade = m @ (l - d)
    
    tokens_from_trade = get_tokens_from_trade(trade, sim_env.all_tokens)
    
    if tokens_from_trade[c.tokens[0]] > 0:
        # This means that I got this token out from the CFMM
        out_token = c.tokens[0]
        in_token = c.tokens[1]
    else: 
        out_token = c.tokens[1]
        in_token = c.tokens[0]
    
    if c.is_transfer: 
        swap_type = 'transfer'
    else: 
        swap_type = 'swap'
        
    swaps.append(Swap(in_token, abs(tokens_from_trade[in_token]), out_token, abs(tokens_from_trade[out_token]), swap_type))

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Feb 03 03:19:36 PM: Your problem has 200 variables, 91 constraints, and 0 parameters.
(CVXPY) Feb 03 03:19:36 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 03 03:19:36 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 03 03:19:36 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Feb 03 03:19:36 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 03 03:19:36 PM: Compiling problem (target solver=CLARABEL)

(CVXPY) Feb 03 03:19:36 PM: Applying reduction Dcp2Cone
(CVXPY) Feb 03 03:19:36 PM: Applying reduction CvxAttr2Constr
(CVXPY) Feb 03 03:19:36 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Feb 03 03:19:36 PM: Applying reduction CLARABEL
(CVXPY) Feb 03 03:19:36 PM: Finished problem compilation (took 3.009e-01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Feb 03 03:19:36 PM: Invoking solver CLARABEL  to obtain a solution.
-------------------------------------------------------------
           Clarabel.rs v0.6.0  -  Clever Acronym              

                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 230
  constraints   =

/home/stephonh/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
 82%|████████▎ | 33/40 [00:09<00:02,  2.74it/s]

Failed for t=2.9747100668324343e-07


 85%|████████▌ | 34/40 [00:10<00:02,  2.69it/s]

...

Failed for t=1.760763014508548e-06


 98%|█████████▊| 39/40 [00:11<00:00,  3.21it/s]

Failed for t=1.982505608670415e-05


100%|██████████| 40/40 [00:12<00:00,  3.24it/s]

Failed for t=0.015989005180591064


My idea for creating a route is to start at the origin token and keep a ledger of what is going on.
* Start with the start token and figure out where the swaps are where that is the in token
* Go through each of the out tokens and figure out where those are going into
* Keep going until there are no swaps left. 

2/3/2024 -> I'm going to try something out where we are going to turn these swaps into a graph for use later. What's useful for this are: 
* Testing out if the graph has cycles. If there are cycles, we do not want to consider it
* Seeing where things end up. We can save things as a node with values and edges. 


In [6]:
# Getting all the unique tokens in the swaps 
unique_tokens = set()
for swap in swaps: 
    unique_tokens.add(swap.in_token)
    unique_tokens.add(swap.out_token)
    
edges = {k: [] for k in unique_tokens}

for swap in swaps:
    edges[swap.in_token].append(swap.out_token)

In [7]:
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]

    if start == end:
        return [path]

    if start not in graph:
        return []

    paths = []
    for neighbor in graph[start]:
        if neighbor not in path:
            new_paths = find_all_paths(graph, neighbor, end, path)
            for new_path in new_paths:
                paths.append(new_path)

    return paths

paths = find_all_paths(edges, 'WETH', 'ATOM')

In [8]:
class Node: 
    
    def __init__(self, token: str, amount: float) -> None:
        self.token = token 
        self.amount = amount
        self.directions = []
        
    def __repr__(self) -> str:
        return f'{self.token}({self.amount})'
    
    def __str__(self) -> str:
        return self.__repr__()
        
class DirectionNode: 
    
    def __init__(self, token: str, amount: float) -> None:
        self.token = token 
        self.amount = amount
        
    def __repr__(self) -> str:
        return f'{self.token}({self.amount})'
    
    def __str__(self) -> str:
        return self.__repr__()

In [9]:
# Creating the nodes from the swaps 
nodes = {token: Node(token, 0) for token in unique_tokens}

target_token = 'ATOM'
for swap in swaps: 
    # Upading the amount of the nodes
    in_token = swap.in_token
    out_token = swap.out_token
    nodes[in_token].amount += swap.in_token_amount
    
    if out_token == target_token:
        nodes[out_token].amount += swap.out_token_amount
    
    # Adding the children
    nodes[in_token].directions.append(Swap(in_token, swap.in_token_amount, out_token, swap.out_token_amount, swap.swap_type))

In [15]:
import queue
starting_token = 'WETH'
target_token = 'ATOM'
starting_node = nodes[starting_token]
visited_nodes = [] 

# Starting the BFS
bfs_queue = queue.SimpleQueue()
bfs_queue.put(starting_node)
directions = []

while not bfs_queue.empty():
    current_node = bfs_queue.get()
    
    # Checking that we have not been here before
    if current_node in visited_nodes: 
        break
    else: 
        visited_nodes.append(current_node)
    
    if (len(current_node.directions) > 0) and (current_node.token != target_token): 
        
        for direction in current_node.directions: 
            next_node = nodes[direction.out_token]
            bfs_queue.put(next_node)
            directions.append(direction)

In [35]:
# Simulating going through the token swap process with logs
token_swap_queue = queue.SimpleQueue()

start_token = 'WETH'
target_token = 'ATOM'
token_swap_queue.put(start_token)

In [36]:
while token_swap_queue.qsize() > 0: 
    # Getting the current token
    current_token = token_swap_queue.get()
    current_amount = nodes[current_token].amount
    
    print(f"Current token: {current_token}")
    print(f'Current amount of {current_token}: {current_amount}')
    
    directions = nodes[current_token].directions
    
    for direction in directions: 
        if direction.swap_type == 'transfer': 
            print(f"Transfering {direction.in_token_amount} {direction.in_token} to {direction.out_token_amount} {direction.out_token}")
        else: 
            print(f"Swapping {direction.in_token_amount} {direction.in_token} for {direction.out_token_amount} {direction.out_token}")
    
    
    print("*" * 50)
    # Look for the swaps that have this token as the input and are not the 
    next_tokens = [x.out_token for x in directions if x.out_token != target_token]
    
    # Append the next token to the queue
    for token in next_tokens: 
        token_swap_queue.put(token)

Current token: WETH
Current amount of WETH: 1999.9999998266978
Swapping 129.360528234321 WETH for 121.93361249438505 CENTAURI/OSMOSIS/SCRT
Transfering 1870.6394715923768 WETH to 1837.8036049994703 ETHEREUM/WETH
**************************************************
Current token: CENTAURI/OSMOSIS/SCRT
Current amount of CENTAURI/OSMOSIS/SCRT: 121.93361239423511
Swapping 121.93361152222951 CENTAURI/OSMOSIS/SCRT for 114.28795352775184 USDC
Swapping 8.720056063330801e-07 CENTAURI/OSMOSIS/SCRT for 6.718211867454115e-08 CENTAURI/OSMOSIS/ATOM
**************************************************
Current token: ETHEREUM/WETH
Current amount of ETHEREUM/WETH: 1837.803604940107
Swapping 957.6574148473323 ETHEREUM/WETH for 883.1715941069597 OSMOSIS/ATOM
Transfering 880.1461900927748 ETHEREUM/WETH to 867.6675777662083 CENTAURI/ETHEREUM/WETH
**************************************************
Current token: USDC
Current amount of USDC: 114.28795347929676
Transfering 114.28795347929676 USDC to 112.3768673196